In [64]:
import pandas as pd
import numpy as np
%matplotlib inline 
import spacy
import re
from functools import reduce
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn import ensemble
from sklearn.model_selection import train_test_split

from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


height has been deprecated.



In [2]:
# Data obtained from Figure Eight.  Contains economic news articles that were categorized as either having information on the US economy or not.
data = pd.read_csv("Economic-News.csv",encoding = "ISO-8859-1")
#link: https://www.figure-eight.com/wp-content/uploads/2016/03/Full-Economic-News-DFE-839861.csv

In [3]:
data.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,positivity,positivity:confidence,relevance,relevance:confidence,articleid,date,headline,positivity_gold,relevance_gold,text
0,842613455,False,finalized,3,12/5/15 17:48,3.0,0.6400,yes,0.640,wsj_398217788,8/14/91,Yields on CDs Fell in the Latest Week,NaN,NaN,NEW YORK -- Yields on most certificates of dep...
1,842613456,False,finalized,3,12/5/15 16:54,NaN,NaN,no,1.000,wsj_399019502,8/21/07,The Morning Brief: White House Seeks to Limit ...,NaN,NaN,The Wall Street Journal Online</br></br>The Mo...
2,842613457,False,finalized,3,12/5/15 1:59,NaN,NaN,no,1.000,wsj_398284048,11/14/91,Banking Bill Negotiators Set Compromise --- Pl...,NaN,NaN,WASHINGTON -- In an effort to achieve banking ...
3,842613458,False,finalized,3,12/5/15 2:19,NaN,0.0000,no,0.675,wsj_397959018,6/16/86,Manager's Journal: Sniffing Out Drug Abusers I...,NaN,NaN,The statistics on the enormous costs of employ...
4,842613459,False,finalized,3,12/5/15 17:48,3.0,0.3257,yes,0.640,wsj_398838054,10/4/02,Currency Trading: Dollar Remains in Tight Rang...,NaN,NaN,NEW YORK -- Indecision marked the dollar's ton...


In [4]:
data.shape

(8000, 15)

In [5]:
# Check for imbalance
data["relevance"].value_counts()


no          6571
yes         1420
not sure       9
Name: relevance, dtype: int64

In [6]:
#Separate data into relevant and not relevant for parsing
relevant = data[data["relevance"] == "yes"]
not_relevant = data[data["relevance"] == "no"]
relevant = relevant.iloc[0:500,]
not_relevant = not_relevant.iloc[0:500,]
yes_text = relevant["text"]
no_text = not_relevant["text"]

In [7]:
#Create list to see samples of data to determine what cleaning is needed
text= data["text"]
text_list= (1,100,500,1000,1500,2250,2500,3000,3350,3400,4500)

In [8]:
# See text to determine cleaning needed.
for headline in text_list:
    print (text[headline])

The Wall Street Journal Online</br></br>The Morning Brief, a look at the day's biggest news, is emailed to subscribers by 7 a.m. every business day. Sign up for the e-mail here.</br></br>On Friday evening, with Congress out of town on its summer recess and Americans heading into a mid-August weekend, the Bush administration sent a message to the states: The federal government will make it tougher for a national children's insurance program to cover the offspring of middle-income families.</br></br>The State Children's Health Insurance Program was created in 1997 to help children whose families couldn't afford insurance but didn't qualify for Medicaid, and administration officials tell the New York Times that the changes are aimed at returning the program to its low- income focus and assuring it didn't become a replacement for private insurance. Administration point man Dennis Smith wrote to state officials saying there would be new restrictions on the District of Columbia and the 18 st

It's no secret that the tea party faithful regard the Obama administration as a Constitution-shredding tyranny. But in a profile of the movement published last week, the New York Times reported the surprising news that many of the protesters have come to this view as a result of their experiences in the recession: "Their families upended by lost jobs, foreclosed homes and depleted retirement funds, they said they wanted to know why it happened and whom to blame."</br></br>An account of a conservative uprising published a few days later by the Washington Post under the headline, "Appalachia is slipping from grip of Democrats," told the story of a hard-bitten congressional district in western Virginia where the response to the recession has been a dramatic swerve to the right.</br></br>The free-market system blunders into recession; its victims flock to the free-market banner. And here we go again.</br></br>The backlash against liberalism has been going on for more than 40 years. It is a

Regan, in his strongest language to date, blamed banks and other financial institutions yesterday for keeping interest rates high while other sectorsof the economy were helping to reduce inflation.</br></br>Regan, insisting that the large federal budget deficits were not responsible for the persistent high interest rates,,, said, there doesnÛªt ÛÏseem to be a good reason-ÛÓor should I say there is no good excuseÛ for those rates to be'.as high as they are, especially with ihe recent declines in inflation.</br></br>He said that if banks set their charges more in line with the declining inflation rateÛÓwhich has dropped below 4 percent from about 13 percent at the start of the Reagan administrationÛÓinterest rates would come down.</br></br>ÛÏOther major sections of the American economy are now moving in accord with the current environment of low inflation,Û Regan told a luncheon of the Chemical Manufacturers Association. ÛÏAnd sooner, I hope, rather than later, the banks and o

In [9]:
# Clean the text
def text_cleaner(text):
    text = text.apply(lambda x:re.sub("</br></br>", "", x))
    text = text.apply(lambda x:re.sub("---", "", x))
    text = text.apply(lambda x:re.sub("--", "", x))
    text = text.apply(lambda x:re.sub("<", "", x))
    text = text.apply(lambda x:re.sub(";", "", x))
    text = text.apply(lambda x:re.sub(";", "", x))
    text = text.apply(lambda x:re.sub("ÛÏ", "", x))
    text = text.apply(lambda x:re.sub("Ûª", "", x))
    text = text.apply(lambda x:re.sub("Û", "", x))
    text = text.apply(lambda x:re.sub("ûó", "", x))
    text = text.apply(lambda x:re.sub("ÛÓ", "", x))
    text = text.apply(lambda x:re.sub("\x89ûó", "", x))
    text = text.apply(lambda x:re.sub("x89ûó", "", x))
    text = text.apply(lambda x:re.sub("\x89ðê", "", x))  
    text = text.apply(lambda x:re.sub("Author:", "", x))
    text = text.apply(lambda x:re.sub("By:", "", x))
    return text
cleaned_text=text_cleaner(text)
yes_cleaned = text_cleaner(yes_text)
no_cleaned = text_cleaner(no_text)
    

In [11]:
# See cleaned text
# See text to determine cleaning needed.
for headline in text_list:
    print (cleaned_text[headline])

The Wall Street Journal OnlineThe Morning Brief, a look at the day's biggest news, is emailed to subscribers by 7 a.m. every business day. Sign up for the e-mail here.On Friday evening, with Congress out of town on its summer recess and Americans heading into a mid-August weekend, the Bush administration sent a message to the states: The federal government will make it tougher for a national children's insurance program to cover the offspring of middle-income families.The State Children's Health Insurance Program was created in 1997 to help children whose families couldn't afford insurance but didn't qualify for Medicaid, and administration officials tell the New York Times that the changes are aimed at returning the program to its low- income focus and assuring it didn't become a replacement for private insurance. Administration point man Dennis Smith wrote to state officials saying there would be new restrictions on the District of Columbia and the 18 states  including California and

It's no secret that the tea party faithful regard the Obama administration as a Constitution-shredding tyranny. But in a profile of the movement published last week, the New York Times reported the surprising news that many of the protesters have come to this view as a result of their experiences in the recession: "Their families upended by lost jobs, foreclosed homes and depleted retirement funds, they said they wanted to know why it happened and whom to blame."An account of a conservative uprising published a few days later by the Washington Post under the headline, "Appalachia is slipping from grip of Democrats," told the story of a hard-bitten congressional district in western Virginia where the response to the recession has been a dramatic swerve to the right.The free-market system blunders into recession its victims flock to the free-market banner. And here we go again.The backlash against liberalism has been going on for more than 40 years. It is as immediate as this morning's n

Regan, in his strongest language to date, blamed banks and other financial institutions yesterday for keeping interest rates high while other sectorsof the economy were helping to reduce inflation.Regan, insisting that the large federal budget deficits were not responsible for the persistent high interest rates,,, said, there doesnt seem to be a good reason-ÛÓor should I say there is no good excuse for those rates to be'.as high as they are, especially with ihe recent declines in inflation.He said that if banks set their charges more in line with the declining inflation rateÛÓwhich has dropped below 4 percent from about 13 percent at the start of the Reagan administrationÛÓinterest rates would come down.Other major sections of the American economy are now moving in accord with the current environment of low inflation, Regan told a luncheon of the Chemical Manufacturers Association. And sooner, I hope, rather than later, the banks and other, financial institutions must do the same.Ho

In [12]:
#Combined rows into one string for NLP parsing.
yes_text_combined = reduce(lambda x, y: x+y, yes_cleaned)
yes_text_combined = yes_text_combined.lower()


no_text_combined = reduce(lambda x, y: x+y, no_cleaned)
no_text_combined = no_text_combined.lower()


In [13]:
no_cleaned.head()

1    The Wall Street Journal OnlineThe Morning Brie...
2    WASHINGTON  In an effort to achieve banking re...
3    The statistics on the enormous costs of employ...
6    TORONTO  Royal Bank of Canada and Bank of Mont...
7    Many people think that the monster of health-c...
Name: text, dtype: object

In [14]:
#Load English Spacy
nlp = spacy.load("en")

In [15]:
#Increase max length(receive error otherwise), the process
nlp.max_length = 11000000
yes_nlp = nlp(yes_text_combined)

In [17]:
no_nlp = nlp(no_text_combined)

In [18]:
yes_sents = [[sent,"Relevant"] for sent in yes_nlp.sents]
no_sents = [[sent,"Not_Relevant"] for sent in no_nlp.sents]
sentences = pd.DataFrame(yes_sents + no_sents)
nlp_text =sentences
sentences.head(10)

,0,1
0,"(new, york, , yields, on, most, certificates,...",Relevant
1,"(two, banks, in, the, banxquote, survey, ,, ci...",Relevant
2,"(against, the, yen, ,, the, dollar, was, tradi...",Relevant
3,"(against, the, swiss, franc, ,, the, dollar, w...",Relevant
4,"(17.weighing, on, the, downside, were, alcoa, ...",Relevant
5,"(hewlett, -, packard, dropped, 21, cents, ,, o...",Relevant
6,"(the, decline, continued, in, new, york, tradi...",Relevant
7,"(lower, bond, prices, translate, into, higher,...",Relevant
8,"(the, bond, market, later, closed, little, -, ...",Relevant
9,"(,, new, york.in, late, new, york, trading, ,,...",Relevant


In [19]:
yes_sents

[[new york  yields on most certificates of deposit offered by major banks dropped more than a tenth of a percentage point in the latest week, reflecting the overall decline in short-term interest rates.on small-denomination, or "consumer," cds sold directly by banks, the average yield on six-month deposits fell to 5.49% from 5.62% in the week ended yesterday, according to an 18-bank survey by banxquote money markets, a wilmington, del., information service.on three-month "consumer" deposits, the average yield sank to 5.29% from 5.42% the week before, according to banxquote.,
  'Relevant'],
 [two banks in the banxquote survey, citibank in new york and corestates in pennsylvania, are paying less than 5% on threemonth small-denomination cds.declines were somewhat smaller on five-year consumer cds, which eased to 7.37% from 7.45%, banxquote said.yields on three-month and six-month treasury bills sold at monday's auction plummeted more than a fifth of a percentage point from the previous we

In [21]:
def word_counter(text, include_stop=True):
        words=[]
        for token in text:
            if not token.is_punct and (not token.is_stop or include_stop):
                words.append(token.text)
        return Counter(words)

In [22]:
word_frequency = word_counter(yes_nlp, include_stop=False).most_common(20)
print(word_frequency)

[("'s", 1008), ('$', 593), ('said', 517), ('market', 475), ('u.s', 471), ('year', 430), (' ', 406), ('prices', 375), ('economy', 369), ('rate', 355), ('rates', 345), ('economic', 337), ('interest', 326), ('fed', 305), ('new', 303), ('stock', 278), ('federal', 271), ('dollar', 265), ('quarter', 259), ('growth', 250)]


In [23]:
word_frequency = word_counter(no_nlp, include_stop=False).most_common(20)
print(word_frequency)

[("'s", 1157), ('$', 635), ('u.s', 454), (' ', 446), ('market', 441), ('said', 441), ('year', 369), ("n't", 357), ('stock', 339), ('new', 307), ('rates', 260), ('investors', 240), ('billion', 233), ('interest', 233), ('economic', 229), ('economy', 221), ('federal', 217), ('rate', 215), ('million', 214), ('mr', 212)]


In [24]:
# Function to count lemmas
def lemma_frequencies(text, include_stop=True):
    lemmas = []
    for token in text:
        if not token.is_punct and (not token.is_stop or include_stop):
            lemmas.append(token.lemma_)
    return Counter(lemmas)

In [26]:
#See most common lemmas
text_lemma_freq = lemma_frequencies(yes_nlp, include_stop=False).most_common(10)
print(text_lemma_freq)

[("'s", 875), ('rate', 700), ('say', 607), ('market', 596), ('$', 593), ('year', 550), ('price', 510), ('stock', 460), ('rise', 438), ('u.s', 414)]


In [27]:
#See most common lemmas
text_lemma_freq = lemma_frequencies(no_nlp, include_stop=False).most_common(10)
print(text_lemma_freq)

[("'s", 1016), ('$', 635), ('say', 601), ('market', 579), ('year', 505), ('stock', 483), ('rate', 475), (' ', 446), ('u.s', 379), ('not', 357)]


In [143]:
#Add text and relevance to dataframe
text_relevance = pd.DataFrame()
text_relevance["text"] = cleaned_text
text_relevance["relevance"] = data["relevance"]

In [173]:
sentences

,0,1
0,"(new, york, , yields, on, most, certificates,...",Relevant
1,"(the, wall, street, journal, onlinethe, mornin...",Not_Relevant
2,"(sign, up, for, the, e, -, mail, here.on, frid...",Not_Relevant


In [28]:
#Bag of words features
def bag_of_words(text):
    allwords = [token.lemma_ for token in text if not token.is_punct and not token.is_stop]
    return [item[0] for item in Counter(allwords).most_common(2000)]

def bow_features(sentences, common_words):
    bow_df = pd.DataFrame(columns=common_words)
    bow_df["text"] = sentences[0]
    bow_df["relevance"] = sentences[1]
    bow_df.loc[:,common_words] = 0
    for i, sentence in enumerate(bow_df["text"]):
        words = [token.lemma_ for token in sentence if (not token.is_punct and not token.is_stop and token.lemma_ in common_words)]
        for word in words:
            bow_df.loc[i, word] +=1
        if i % 500 == 0:
            print("Processing row {}".format(i))
    return bow_df

In [29]:
yes_words = bag_of_words(yes_nlp)
no_words = bag_of_words(no_nlp)
common_words = set(yes_words + no_words)

In [30]:
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000
Processing row 5500
Processing row 6000
Processing row 6500


,open,0.3,one,structure,pace,wind,bidding,wage,imf,largely,tone,blog,office,mix,double,set,end,knowledge,trading,rapidly,client,l.,flow,send,saving,hill,young,line,treatment,delivery,banxquote,texas,cut,design,follow,react,manage,margin,confidence,mid,likelihood,production,web,call,fight,mark,efficiency,dome,perceive,man,nationwide,room,85,condition,jerry,error,paulsen,firm,enjoy,prudential,gear,auction,trust,consistent,testimony,higher,fall,chairman,crisis,crude,fail,trigger,recruit,memory,run,manufacturer,proposal,statement,consequence,massachusetts,load,staff,5.3,dip,ally,file,12,eastern,7/8,17,part,fee,shortage,doctor,commerce,interview,resolution,argue,economically,romney,spokesman,path,anticipation,blackstone,insider,surplus,1990,clinton,internet,icahn,violate,stress,britain,hot,unit,pledge,0.9,depressed,legislation,1970,important,driver,participation,brady,credit,usually,gamble,extraordinary,jone,policy,1.2,7.3,succeed,friday,shopper,fourth,persist,aim,troublesome,high.the,plunge,downward,diluted,quickly,2.8,fastenal,weather,scrutiny,plague,wci,exist,king,easy,restructuring,book,federal,dump,rates.the,administration,publication,maintain,restore,pull,table,stable,gyration,trouble,'s,spur,believe,expensive,earlier,subdued,19,pick,nasdaq,typical,factory,spring,5.8,weak,ahead,75,early,mayor,pend,aide,unemployed,degree,damage,core,bleak,consistently,gold,marketing,easing,like,initial,belief,troubled,debt,e.,hint,group,suddenly,relate,quote,capitalization,pound,anecdotal,1989,comment,output,rent,mutual,gauge,equity,freeze,key,associate,2006,minister,thing,david,period,impact,normal,feeling,old,card,broker,basis,parent,popular,fine,mainly,park,8.9,brand,edge,general,level,court,number,offering,hourly,3.3,sales.the,say,lender,...,lawyer,year.the,treasury,extend,trend,tap,consecutive,55,senator,employment,report,steady,rupee,make,expenditure,franc,apply,standard,clip,similar,outside,regard,era,medical,compromise,1986,get,course,fare,break,currency,neighborhood,truck,"20,000",pharmaceutical,u.,longer,crash,3/4,1.4,foot,incentive,w.,leap,vehicle,1981,agreement,available,freddie,blow,loan,special,describe,war,patient,nikkei,suffer,supplier,visa,los,20,evidence,child,nervous,larry,people,3.7,refund,writer,side,inflationary,type,1.9,city,strategy,age,hewlett,accord,account,aggressive,smith,euro,back,let,calculation,participate,retire,officer,project,effectively,operating,television,worth,speculation,presidential,red,listing,decision,facebook,hearing,sovereign,thousand,inflow,welfare,poise,tomorrow,expect,relief,valley,1/2,cliff,tenth,originally,barometer,act,"200,000",senate,highlight,notably,specialist,deserve,official,toll,railroad,attempt,agent,money,limited,household,marginal,ward,steam,aircraft,commentary,landing,nearly,season,barely,persistent,uncertain,hardly,lead,tech,reminder,teacher,admit,homeowner,indian,kansa,gasoline,september,school,28,drag,2.6,sign,measure,concession,civil,communication,obligation,watcher,situation,a.m.,daniel,reform,property,partner,joel,prominent,basic,significant,furniture,town,90,receipt,check,address,4.4,index,recession,clear,paul,200,coverage,harsh,rank,dr,21,11.5,hybrid,months.the,january,favor,access,1992,offer,1994,banking,row,especially,orient,0.5,institution,bell,export,emission,harris,contract,halve,emergency,direction,bother,ago,lay,strongly,begin,manager,potentially,aug,pass,dodge,co.,30,independence,meet,appeal,38,speak,2.5,revision,brother,unusually,gradually,ticket,exceed,russell,reaction,rush,statistical,story,black,guarantee,ipo,greenspan,worry,afternoon,actually,text,relevance
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [31]:
word_counts.head()

,open,0.3,one,structure,pace,wind,bidding,wage,imf,largely,tone,blog,office,mix,double,set,end,knowledge,trading,rapidly,client,l.,flow,send,saving,hill,young,line,treatment,delivery,banxquote,texas,cut,design,follow,react,manage,margin,confidence,mid,likelihood,production,web,call,fight,mark,efficiency,dome,perceive,man,nationwide,room,85,condition,jerry,error,paulsen,firm,enjoy,prudential,gear,auction,trust,consistent,testimony,higher,fall,chairman,crisis,crude,fail,trigger,recruit,memory,run,manufacturer,proposal,statement,consequence,massachusetts,load,staff,5.3,dip,ally,file,12,eastern,7/8,17,part,fee,shortage,doctor,commerce,interview,resolution,argue,economically,romney,spokesman,path,anticipation,blackstone,insider,surplus,1990,clinton,internet,icahn,violate,stress,britain,hot,unit,pledge,0.9,depressed,legislation,1970,important,driver,participation,brady,credit,usually,gamble,extraordinary,jone,policy,1.2,7.3,succeed,friday,shopper,fourth,persist,aim,troublesome,high.the,plunge,downward,diluted,quickly,2.8,fastenal,weather,scrutiny,plague,wci,exist,king,easy,restructuring,book,federal,dump,rates.the,administration,publication,maintain,restore,pull,table,stable,gyration,trouble,'s,spur,believe,expensive,earlier,subdued,19,pick,nasdaq,typical,factory,spring,5.8,weak,ahead,75,early,mayor,pend,aide,unemployed,degree,damage,core,bleak,consistently,gold,marketing,easing,like,initial,belief,troubled,debt,e.,hint,group,suddenly,relate,quote,capitalization,pound,anecdotal,1989,comment,output,rent,mutual,gauge,equity,freeze,key,associate,2006,minister,thing,david,period,impact,normal,feeling,old,card,broker,basis,parent,popular,fine,mainly,park,8.9,brand,edge,general,level,court,number,offering,hourly,3.3,sales.the,say,lender,...,lawyer,year.the,treasury,extend,trend,tap,consecutive,55,senator,employment,report,steady,rupee,make,expenditure,franc,apply,standard,clip,similar,outside,regard,era,medical,compromise,1986,get,course,fare,break,currency,neighborhood,truck,"20,000",pharmaceutical,u.,longer,crash,3/4,1.4,foot,incentive,w.,leap,vehicle,1981,agreement,available,freddie,blow,loan,special,describe,war,patient,nikkei,suffer,supplier,visa,los,20,evidence,child,nervous,larry,people,3.7,refund,writer,side,inflationary,type,1.9,city,strategy,age,hewlett,accord,account,aggressive,smith,euro,back,let,calculation,participate,retire,officer,project,effectively,operating,television,worth,speculation,presidential,red,listing,decision,facebook,hearing,sovereign,thousand,inflow,welfare,poise,tomorrow,expect,relief,valley,1/2,cliff,tenth,originally,barometer,act,"200,000",senate,highlight,notably,specialist,deserve,official,toll,railroad,attempt,agent,money,limited,household,marginal,ward,steam,aircraft,commentary,landing,nearly,season,barely,persistent,uncertain,hardly,lead,tech,reminder,teacher,admit,homeowner,indian,kansa,gasoline,september,school,28,drag,2.6,sign,measure,concession,civil,communication,obligation,watcher,situation,a.m.,daniel,reform,property,partner,joel,prominent,basic,significant,furniture,town,90,receipt,check,address,4.4,index,recession,clear,paul,200,coverage,harsh,rank,dr,21,11.5,hybrid,months.the,january,favor,access,1992,offer,1994,banking,row,especially,orient,0.5,institution,bell,export,emission,harris,contract,halve,emergency,direction,bother,ago,lay,strongly,begin,manager,potentially,aug,pass,dodge,co.,30,independence,meet,appeal,38,speak,2.5,revision,brother,unusually,gradually,ticket,exceed,russell,reaction,rush,statistical,story,black,guarantee,ipo,greenspan,worry,afternoon,actually,text,relevance
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [34]:
rfc = ensemble.RandomForestClassifier()
y = word_counts['relevance']
x = np.array(word_counts.drop(['text','relevance'], 1))

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=0)
train = rfc.fit(x_train, y_train)

print('Training set score:', rfc.score(x_train, y_train))
print('\nTest set score:', rfc.score(x_test, y_test))
cross = cross_val_score(rfc,x_train,y_train, cv=3)
print(cross)

Training set score: 0.977871705619

Test set score: 0.630499627144
[ 0.58314691  0.5950783   0.59701493]


In [87]:
#Create vectorizer
x_train, x_test = train_test_split(cleaned_text, test_size=0.4, random_state=0)

vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english', lowercase=True, use_idf=False, norm=u'l1', smooth_idf=True)

In [88]:
text_tfidf = vectorizer.fit_transform(cleaned_text)

x_train_tfidf, x_test_tfidf = train_test_split(text_tfidf, test_size=.4, random_state=42)

x_train_tfidf_csr = x_train_tfidf.tocsr()

In [59]:
#print(text_tfidf.shape)
print(text_tfidf[0])

  (0, 14056)	0.0533267427747
  (0, 22954)	0.0733531378743
  (0, 22946)	0.134169199395
  (0, 4371)	0.0973813885056
  (0, 6438)	0.0804662474129
  (0, 14437)	0.0709389407179
  (0, 12849)	0.0471388789863
  (0, 2911)	0.151261203762
  (0, 7288)	0.0577709350553
  (0, 20656)	0.104642129895
  (0, 15239)	0.117124830396
  (0, 15647)	0.0915432542
  (0, 12126)	0.0570184602685
  (0, 22482)	0.153177066248
  (0, 17021)	0.0797992353488
  (0, 14736)	0.0615694298824
  (0, 6198)	0.049053465388
  (0, 18723)	0.0503253657969
  (0, 20667)	0.046184990936
  (0, 16694)	0.0351717240093
  (0, 19090)	0.107790322412
  (0, 6398)	0.271499387126
  (0, 5366)	0.147845416169
  (0, 4311)	0.336671363522
  (0, 19218)	0.130133186875
  :	:
  (0, 18526)	0.0608733633955
  (0, 18132)	0.0859915587706
  (0, 566)	0.0671414792741
  (0, 756)	0.0740036134178
  (0, 4658)	0.103992153135
  (0, 5554)	0.125163846754
  (0, 15215)	0.092480438087
  (0, 15129)	0.0722346127075
  (0, 6202)	0.0626768894545
  (0, 19249)	0.075702267856
  (0, 19092)	

In [41]:
cleaned_text.head()

0    NEW YORK  Yields on most certificates of depos...
1    The Wall Street Journal OnlineThe Morning Brie...
2    WASHINGTON  In an effort to achieve banking re...
3    The statistics on the enormous costs of employ...
4    NEW YORK  Indecision marked the dollar's tone,...
Name: text, dtype: object

In [39]:
x_train.shape

(4800,)

In [89]:
n = x_train_tfidf_csr.shape[0]

tfidf_bypara = [{} for _ in range(0,n)]

terms = vectorizer.get_feature_names()

for i, j in zip(*x_train_tfidf_csr.nonzero()):
    tfidf_bypara[i][terms[j]] = x_train_tfidf_csr[i, j]




In [44]:
print('Original sentence:', x_train.iloc[0])
print('Tf_idf vector:', tfidf_bypara[0])

Original sentence: Charles County Chamber of Commerce: 301-932-6500, offering business assistance, computers, phones and other help.The People's Place Comprehensive Service Center: Open 7 a.m. to 8 p.m. at Somer's House, County Government Building, La Plata.Federal Emergency Management Agency: 800-621-FEMA. Individuals, families and business owners seeking federal disaster aid may register by calling FEMA.Maryland Emergency Management Agency: 877-MEMA-USA or 410-517- 3600. Charles County MEMA: 301-645-0630. Calvert County MEMA: 410- 535-1623.Maryland Small Business Development Center: 888-246-6736, assists companies with the process of applying for Small Business Administration low-interest loans.
Tf_idf vector: {'stockpiles': 0.16384898252927932, 'pall': 0.17072411592059206, '845': 0.16818128472647434, 'glum': 0.16818128472647434, 'beaten': 0.146124741122754, 'bounce': 0.14079525115632191, 'readings': 0.13296044366704837, 'experiencing': 0.13708515664572449, 'worsened': 0.155759683538

In [45]:
print('Original sentence:', x_train.iloc[1])
print('Tf_idf vector:', tfidf_bypara[1])

Original sentence: Regarding Holman Jenkins, Jr.'s insightful Nov. 20 Business World column "How the GOP Should Fix ObamaCare": His idea not only highlights a solution but also the GOP's biggest problemto wit, Republicans do not know how to communicate effectively. Does anyone realize that there already is a bill in the House (H.R. 3121) that allows for insurance to be sold across state lines and provides additional remedies to address our nation's health-care problems?You do not solve such a big problem by simply introducing a bill that addresses the problem. You investigate how to let everyone in the country know that such a bill exists by giving it a catchy name (I propose "Organic Care," a grass-roots solution), a slogan, its own website with bullet points describing its main points, plus YouTube videos, jingles, songs, stand-up comedy routines, social-media exposure, etc. That requires getting creative. Republicans think that their philosophy, their message, is adequately promulga

In [61]:
x_train.head()

4136    Charles County Chamber of Commerce: 301-932-65...
3357    Regarding Holman Jenkins, Jr.'s insightful Nov...
7546    said yesterday its open market committee will ...
7426    Pr/ncessMarce/fa Borgfiese Brings yvus H&yspee...
7382    Basic subway and bus fares could rise by as mu...
Name: text, dtype: object

In [62]:
terms = vectorizer.get_feature_names()

In [63]:
print(terms)

['00', '000', '001', '003', '008', '009', '01', '012', '013', '014', '0156', '017', '0188', '02', '021', '022', '024', '025', '027', '03', '030', '0304', '037', '04', '040', '043', '045', '046', '047', '049', '05', '050', '052', '059', '06', '060', '0625', '064', '068', '069', '07', '070', '075', '077', '08', '080', '081', '083', '084', '085', '088', '089', '09', '090', '091', '096', '0f', '10', '100', '1000', '10000', '1008', '100th', '101', '1010wallstreet', '10174', '1019', '101st', '102', '10244', '103', '10353', '1039', '104', '1040', '10402', '1046', '105', '1057', '106', '107', '1070', '1074', '1076', '108', '1084', '109', '1094', '1096', '10th', '11', '110', '11000', '11010', '111', '1112', '1118', '112', '1121', '113', '1130', '1134', '1135', '114', '115', '1151', '116', '1160', '117', '1170', '11722', '1175', '1178', '118', '1181', '1188', '119', '11th', '12', '120', '12000', '1202', '121', '122', '12258', '123', '1235', '124', '125', '126', '1261', '127', '1272', '12786', '1

In [90]:
svd = TruncatedSVD(1000)
lsa = make_pipeline(svd, Normalizer(copy=False))
# Run SVD on the training data, then project the training data.
x_train_lsa = lsa.fit_transform(x_train_tfidf)

variance_explained=svd.explained_variance_ratio_
total_variance = variance_explained.sum()
print("Percent variance captured by all components:",total_variance*100)

#Looking at what sorts of paragraphs our solution considers similar, for the first five identified topics
paras_by_component=pd.DataFrame(x_train_lsa,index=x_train)
for i in range(5):
    print('Component {}:'.format(i))
    print(paras_by_component.loc[:,i].sort_values(ascending=False)[0:10])

Percent variance captured by all components: 78.167572759
Component 0:
text
I recently had the privilege of working closely with two men of uncommon courage  Andrew F. Brimmer, former chair of the D.C. financial control board, and Julius W. Becton Jr., former chief executive officer and superintendent of the D.C. public schools.Before taking up their work in the District, these men had climbed the heights of their chosen professions. Brimmer was one of the leading economists in the United States and a member of the Federal Reserve Board. Becton had retired, after 40 years of service, as a U.S. Army lieutenant general. Both Brimmer and Becton are members of what Thomas Jefferson called that "natural aristocracy," based not upon birth but upon "virtue and talents."Brimmer and Becton volunteered for arduous and time-consuming positions in our community in a time of troubles. They did this not for recognition, of which each already had received a full measure, but because they hoped to mak

Name: 0, dtype: float64
Component 1:
text
It was a bumpy year for So Others Might Eat, the District-based charity known as SOME, which provides food, clothing, medical care and other services to those in need.Just months before the charity was to open a glossy building in Northwest Washington, the Sept. 11 attacks occurred, and SOME's donations plummeted 30 percent as Americans turned their attention - - and wallets  to that cause. Then came the anthrax postal scare, and the charity group's mail dried up for several weeks.For a nonprofit organization that gets more than half its funding in small donations through the mail, it was a staggering one-two punch, said development coordinator Shonna Milliken Humphrey.But last week, the 30-year-old organization finally got some good news. Employees participating in the workplace fundraising campaign of the United Way of the National Capital Area had designated $1.4 million for SOME  a whopping 40 percent more than in the previous year. The new

Name: 1, dtype: float64
Component 2:
text
Robert L. Ehrlich Jr. said Tuesday that he will try to win his old job back in November, confirming plans for a much-anticipated rematch with Maryland Gov. Martin O'Malley, who defeated him in a bitter contest four years ago.A formal announcement, planned for next Wednesday in Montgomery County, will set up a rematch between two of the state's dominant political personalities.Both are fierce competitors. Their 2006 race was highly negative and at times intensely personal, with Ehrlich (R) repeatedly calling O'Malley "a whiner" and running ads that highlighted the homicide rate and struggling schools in Baltimore, where O'Malley was mayor. O'Malley (D) accused Ehrlich of dirty tricks and portrayed him as cozy with energy lobbyists and an unpopular President George W. Bush.Ehrlich said Tuesday that he plans to run a forward-looking campaign, disputing characterizations of the race as a grudge match."The last thing that people want to see is a sch

Name: 2, dtype: float64
Component 3:
text
FedEx Corp. threw some cold water on hopes that holiday shoppers would give the uncertain economic outlook a boost.The Memphis, Tenn., shipping giant  regarded as an economic gauge because it handles an average of more than six million packages a day across the world  reduced its profit target for the year because of an economic outlook pinched by the weak U.S. housing market and higher energy prices. The company said that as a result, it would ratchet back on capital spending."We believe U.S. [gross domestic product] will grow less than 3%" for the rest of FedEx's fiscal year, which ends May 30, said FedEx Chief Financial Officer Alan B. Graf Jr. He said the company lowered its profit guidance "almost entirely based on freight market" and economic forecasts.The reduced outlook overshadowed FedEx's results for the first quarter ended Aug. 31, which rose 4% from a year earlier. At 4 p.m. in New York Stock Exchange composite trading, shares of Fe

Name: 3, dtype: float64
Component 4:
text
FOR A TIME, there was a reassuring story to tell about the stock market and the economy. Stocks rose in a speculative frenzy, and then fell. But the U.S. economy recovered surprisingly quickly from Sept. 11, aided by the irrepressible American impulse to shop, plus a round of interest-rate cutting. The market was sick. The economy wasn't. Silicon Valley paper millionaires stopped buying Porsches. The rest of us kept buying Chevrolets.The magnitude, the speed and the circumstances of the stock-market decline are threatening that comforting story. Unless the market touches bottom soon  and despite yesterday's spectacular bounce in stock prices, there is no assurance of that  it is going to yank the economy down with it.We may not sink back into recession. But if we have a few more bad weeks on Wall Street, you can rip up those forecasts about the economy growing at a 3.5% annual pace in the second half. Instead, the economy will slow so much that

Name: 4, dtype: float64
